In [86]:
import sys
import os
import pymongo
from pymongo import MongoClient
import pandas as pd
import sqlite3
import importlib
import sql_conn
importlib.reload(sql_conn)

sys.path.append(os.path.abspath('/workspaces/projeto_ing/utils/shered_utils/'))
from sql_conn import conectar_bronze


In [ ]:
db_sqlite = "bnc_log_igtao.db"
# db_sqlite = "banco_cep.db"
conectar_bronze(db_sqlite)

conectado ao sqlite bronze: banco_cep.db
Tabelas no banco de dados:
tabela_cep


(<sqlite3.Connection at 0x702a8c2e4640>,
 '/workspaces/projeto_ing/database/sqlite/bronze/banco_cep.db')

In [87]:
db_sqlite = "bnc_log_igtao.db"

conn, cursor, caminho_bronze, consulta, query = conectar_bronze(db_sqlite, tabela= "tb_log_igtao")
# print(bronze)

conectado ao sqlite bronze: bnc_log_igtao.db
Tabelas no banco de dados:
tabela_log_igtao
tb_log_igtao
('banco_logigtao', 'logigtao', 'arquivo_logigtao_20250224_104223.txt', 'txt', 20250224142210, '2025-02-24')
('banco_teste', 'teste', 'arquivo_teste_20250224_104223.txt', 'txt', 20250224142510, '2025-02-24')
('banco_teste', 'tabela_teste', 'arquivo_teste_20250224_104223.txt', 'txt', 20250224145404, '2025-02-24')
('banco_teste', 'tabela_teste', 'arquivo_teste_20250224_104223.txt', 'txt', 20250224145507, '2025-02-24')
('banco_teste', 'tabela_teste', 'arquivo_teste_20250224_104223.txt', 'txt', 20250224145518, '2025-02-24')
('banco_teste', 'tabela_teste', 'arquivo_teste_20250224_104223.txt', 'txt', 20250224145753, '2025-02-24')
('banco_mairipora', 'tabela_mairipora', 'arquivo_mairipora_20250224_104223.txt', 'txt', 20250224150135, '2025-02-24')
('banco_weather', 'tabela_weather', 'resultado_weather_20250108_163946.txt', 'txt', 20250225184651, '2025-02-25')


In [88]:
df_brnz = pd.read_sql_query(query, conn)
print(df_brnz)


        nome_banco       nome_tabela                           nome_arquivo  \
0   banco_logigtao          logigtao   arquivo_logigtao_20250224_104223.txt   
1      banco_teste             teste      arquivo_teste_20250224_104223.txt   
2      banco_teste      tabela_teste      arquivo_teste_20250224_104223.txt   
3      banco_teste      tabela_teste      arquivo_teste_20250224_104223.txt   
4      banco_teste      tabela_teste      arquivo_teste_20250224_104223.txt   
5      banco_teste      tabela_teste      arquivo_teste_20250224_104223.txt   
6  banco_mairipora  tabela_mairipora  arquivo_mairipora_20250224_104223.txt   
7    banco_weather    tabela_weather  resultado_weather_20250108_163946.txt   

  tipo_arquivo   data_inclusao    dt_igtao  
0          txt  20250224142210  2025-02-24  
1          txt  20250224142510  2025-02-24  
2          txt  20250224145404  2025-02-24  
3          txt  20250224145507  2025-02-24  
4          txt  20250224145518  2025-02-24  
5          txt  20

In [77]:
df = pd.DataFrame(bronze)
display(df)

,0
0,<sqlite3.Connection object at 0x702a8c32de40>
1,<sqlite3.Cursor object at 0x702a8c2f7ec0>
2,/workspaces/projeto_ing/database/sqlite/bronze...
3,"[(banco_logigtao, logigtao, arquivo_logigtao_2..."
